## Read data and check data quality

In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import pandas as pd
import numpy as np
from src.paths import RAW_DATA_DIR

In [136]:
# Read data
data = pd.read_csv(RAW_DATA_DIR / 'data_pricing_challenge.csv')

In [137]:
# Check a sample of the data
data.head()

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at
0,BMW,118,140411,100,2/1/2012,diesel,black,convertible,True,True,False,False,True,True,True,False,11300,1/1/2018
1,BMW,M4,13929,317,4/1/2016,petrol,grey,convertible,True,True,False,False,False,True,True,True,69700,2/1/2018
2,BMW,320,183297,120,4/1/2012,diesel,white,convertible,False,False,False,False,True,False,True,False,10200,2/1/2018
3,BMW,420,128035,135,7/1/2014,diesel,red,convertible,True,True,False,False,True,True,True,True,25100,2/1/2018
4,BMW,425,97097,160,12/1/2014,diesel,silver,convertible,True,True,False,False,False,True,True,True,33400,4/1/2018


In [138]:
# Get data shape
data.shape

(4843, 18)

In [139]:
# Check data types
data.dtypes 

maker_key            object
model_key            object
mileage               int64
engine_power          int64
registration_date    object
fuel                 object
paint_color          object
car_type             object
feature_1              bool
feature_2              bool
feature_3              bool
feature_4              bool
feature_5              bool
feature_6              bool
feature_7              bool
feature_8              bool
price                 int64
sold_at              object
dtype: object

In [140]:
# Check number of nulls in columns.
data.isnull().sum()

maker_key            0
model_key            0
mileage              0
engine_power         0
registration_date    0
fuel                 0
paint_color          0
car_type             0
feature_1            0
feature_2            0
feature_3            0
feature_4            0
feature_5            0
feature_6            0
feature_7            0
feature_8            0
price                0
sold_at              0
dtype: int64

There are no null values in the data.

In [141]:
# Check inf values in the data
data[data.isin([np.inf, -np.inf])].sum()

maker_key              0
model_key              0
mileage              0.0
engine_power         0.0
registration_date      0
fuel                   0
paint_color            0
car_type               0
feature_1              0
feature_2              0
feature_3              0
feature_4              0
feature_5              0
feature_6              0
feature_7              0
feature_8              0
price                0.0
sold_at                0
dtype: object

There are no infinite values in the data.

In [142]:
# Find duplicated rows
data[data.duplicated()]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at


There are no duplicates.

In [143]:
# Count unique values in each column
data.nunique()

maker_key               1
model_key              75
mileage              4786
engine_power           61
registration_date     199
fuel                    4
paint_color            10
car_type                8
feature_1               2
feature_2               2
feature_3               2
feature_4               2
feature_5               2
feature_6               2
feature_7               2
feature_8               2
price                 437
sold_at                 9
dtype: int64

From the number of unique feature values we can observe that:
- There is only one maker (BWM). So this feature will not give information to the models.
- There are 199 different registration dates.
- feature_1 to feature_8 are binary variables.
- The auction happened in 9 different dates.

In [144]:
# Convert date columns to datetime
data['registration_date'] = pd.to_datetime(data['registration_date'])
data['sold_at'] = pd.to_datetime(data['sold_at'])

In [145]:
# Check range of registration dates
data['registration_date'].min(), data['registration_date'].max()

(Timestamp('1990-03-01 00:00:00'), Timestamp('2017-11-01 00:00:00'))

In [146]:
# Check range of solt_at dates
data['sold_at'].min(), data['sold_at'].max()

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-09-01 00:00:00'))

Based on the date ranges, the oldest car is at least 27 years old, and the newest less than 1 year old. This suggest that the age of the car will be a variable worth considering.

In [147]:
# Check unique month days in registration_date
print(data['registration_date'].dt.day.nunique())

1


In [148]:
# Check unique month days in sold_at
print(data['sold_at'].dt.day.nunique())

1


Since the only day of the month in all of the dates is day 1, the smallest granularity for days is monthly.

Based on dates granularity and the observation that car ages range from less than a year to at least 27 years old, the "age_in_months_when_sold" variable will be calculated.

In [149]:
data['age_in_months_when_sold'] = (data['sold_at'].dt.to_period('M') - data['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [150]:
data[['registration_date', 'sold_at', 'age_in_months_when_sold']].head()

,registration_date,sold_at,age_in_months_when_sold
0,2012-02-01,2018-01-01,71
1,2016-04-01,2018-02-01,22
2,2012-04-01,2018-02-01,70
3,2014-07-01,2018-02-01,43
4,2014-12-01,2018-04-01,40


In [151]:
# Describe the numeric columns in the data
data.describe()

,mileage,engine_power,registration_date,price,sold_at,age_in_months_when_sold
count,4.843000e+03,4843.00000,4843,4843.000000,4843,4843.000000
mean,1.409628e+05,128.98823,2012-11-22 14:51:42.704934912,15828.081767,2018-04-29 08:15:03.902539520,65.186042
min,-6.400000e+01,0.00000,1990-03-01 00:00:00,100.000000,2018-01-01 00:00:00,7.000000
25%,1.029135e+05,100.00000,2012-07-01 00:00:00,10800.000000,2018-03-01 00:00:00,49.000000
50%,1.410800e+05,120.00000,2013-07-01 00:00:00,14200.000000,2018-05-01 00:00:00,58.000000
75%,1.751955e+05,135.00000,2014-04-01 00:00:00,18600.000000,2018-07-01 00:00:00,70.000000
max,1.000376e+06,423.00000,2017-11-01 00:00:00,178500.000000,2018-09-01 00:00:00,337.000000
std,6.019674e+04,38.99336,NaN,9220.285684,NaN,30.466419


Looking at the description of numerical features, hints that there are possibly erroneous observations. For example:
- a car with -64 miles,
- a car with 0 (I assume hp) engine power, 
- and a car that costed 100 (I assume USD)

Therefore, data needs some further cleaning.

## Find odd observations and remove incorrect observations

### Check mileage

In [152]:
data[data['mileage'] < 0]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold
2938,BMW,640 Gran Coupé,-64,230,2017-03-01,diesel,black,sedan,True,True,False,True,False,False,True,True,75000,2018-04-01,13


In [153]:
len(data[data['model_key'] == '640 Gran Coupé'])

19

There is a 640 Gran Coupé with negative mileage which is not possible. Also, there are other 18 cars of the same model key, so this row will be removed.

In [154]:
data[data['mileage'] > 1e6]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold
3732,BMW,118,1000376,90,2005-03-01,diesel,black,subcompact,True,False,False,False,False,False,True,False,1400,2018-06-01,159


This is a 13 year-old car (159 months). Although strange, it is not impossible that it it has driven this many miles (about 210 in average daily). So this observation is not recommended to be removed.

In [155]:
# Remove row with index 2938 (begative mileage)
data = data.drop(index=2938)

In [156]:
data[data['mileage'] < 0]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold


### Check engine power

In [157]:
data[data['engine_power'] < 70]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold
1796,BMW,i3,152328,25,2014-02-01,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,True,20300,2018-04-01,50
1925,BMW,i3,152470,25,2014-02-01,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,True,15500,2018-03-01,49
2390,BMW,318,170529,66,2000-05-01,diesel,silver,hatchback,False,False,True,False,False,False,False,False,1000,2018-08-01,219
2771,BMW,316,146951,66,1990-03-01,petrol,white,sedan,False,False,False,False,False,False,False,False,1300,2018-04-01,337
3765,BMW,X1,81770,0,2014-10-01,diesel,white,suv,False,False,False,False,False,False,False,False,11700,2018-03-01,41


Check how many other cars are there with the same model key.

In [158]:
len(data[data['model_key'] == 'X1'])

275

In [159]:
len(data[data['model_key'] == 'i3'])

6

In [160]:
len(data[data['model_key'] == '318'])

569

In [161]:
len(data[data['model_key'] == '316'])

235

Describe engine power column for i3, 318, and 316 without suspicious observations.

In [162]:
data[data['model_key'] == 'i3'].drop(index=[1796, 1925])['engine_power'].describe()

count      4.0
mean      87.5
std       25.0
min       75.0
25%       75.0
50%       75.0
75%       87.5
max      125.0
Name: engine_power, dtype: float64

In [163]:
data[data['model_key'] == '316'].drop(index=[2771])['engine_power'].describe()

count    234.000000
mean      84.952991
std        1.511732
min       75.000000
25%       85.000000
50%       85.000000
75%       85.000000
max      100.000000
Name: engine_power, dtype: float64

In [164]:
data[data['model_key'] == '318'].drop(index=[2390])['engine_power'].describe()

count    568.000000
mean     101.276408
std        4.240063
min       85.000000
25%      100.000000
50%      100.000000
75%      105.000000
max      110.000000
Name: engine_power, dtype: float64

There is a wrong observation with 0 engine power for an X1 which is a SUV. This is impossible. Since there are more than 200 other X1 this observation can be removed.

Very likely the engine power of 25 (hp) for two i3 is wrong. These cars should have 75 (hp) engine power, so this is probably a typo. Since there are very few other i3 cars, data imputation might be a better alternative here to dropping the records. According to the majority of values, a good candidate value for imputation is 75.

Regarding models 316 and 318, since there are more than 200 hundred other observations with the same model, and these cars typically have at least 75 of power engine, these records can also be deleted.

In [165]:
# Remove 0 power observation
data = data.drop(index=[3765])

In [166]:
data[data['engine_power'] <= 0]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold


In [167]:
# Replace engine power value for i3 model
data.loc[1796, 'engine_power'] = 75
data.loc[1925, 'engine_power'] = 75

In [168]:
data[data['model_key'] == 'i3']

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold
1796,BMW,i3,152328,75,2014-02-01,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,True,20300,2018-04-01,50
1895,BMW,i3,26542,75,2014-03-01,electro,grey,hatchback,False,True,False,False,False,False,True,True,20900,2018-02-01,47
1925,BMW,i3,152470,75,2014-02-01,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,True,15500,2018-03-01,49
2317,BMW,i3,78740,75,2014-04-01,electro,grey,hatchback,True,True,False,False,False,False,True,False,20800,2018-07-01,51
2363,BMW,i3,46359,125,2014-01-01,electro,white,hatchback,True,True,False,False,False,False,True,True,21200,2018-08-01,55
3690,BMW,i3,6572,75,2015-11-01,hybrid_petrol,black,subcompact,True,True,True,False,False,False,True,True,28000,2018-03-01,28


In [169]:
# Remove low power 316 and 318 observations
data = data.drop(index=[2390, 2771])

In [170]:
data[data['engine_power'] <= 66]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold


## Check price

Are there cars that sold at unusually low prices?

In [171]:
unusually_cheap_cars = data[(data['price'] < 1000)]

In [172]:
unusually_cheap_cars.describe()

,mileage,engine_power,registration_date,price,sold_at,age_in_months_when_sold
count,62.000000,62.000000,62,62.000000,62,62.000000
mean,194706.870968,110.451613,2008-05-04 23:36:46.451612928,646.774194,2018-04-09 12:23:13.548387072,119.161290
min,79685.000000,75.000000,1994-01-01 00:00:00,100.000000,2018-01-01 00:00:00,34.000000
25%,131414.000000,90.250000,2000-03-01 00:00:00,500.000000,2018-03-01 00:00:00,45.000000
50%,165244.500000,105.000000,2013-05-01 00:00:00,800.000000,2018-04-01 00:00:00,58.500000
75%,230499.500000,131.250000,2014-06-01 00:00:00,800.000000,2018-06-01 00:00:00,217.500000
max,439060.000000,190.000000,2015-05-01 00:00:00,900.000000,2018-09-01 00:00:00,291.000000
std,87350.448051,25.376947,NaN,260.967771,NaN,90.298811


There were 62 cars sold at less than 1,000 which is very unusual. Looking at a description of the numerical features of these case we find:
- The newest car sold at this price was less than 3 years old. This seems unusual.
- The oldest car was 24 years old.
- Minimum and max mileage seem sensible.

Although these prices are unusual, maybe some of them are more common suggesting it's not necessarily a mistake.

In [173]:
# Print value counts for price in the unusually cheap cars
unusually_cheap_cars['price'].value_counts()

price
800    36
100     7
400     5
900     4
700     4
200     3
500     2
600     1
Name: count, dtype: int64

Based on the above value counts, we can conclude that prices 800 and above are not necessarily unusual. So these will be kept.

Prices below 800 seem more unusual and might correspond to cars with important damages. Since cars with engine damages were removed before, these other cars with important damages could be removed as well and maybe priced with another strategy.

In [174]:
# Print value counts for model_key in unusually cheap cars
unusually_cheap_cars['model_key'].value_counts().sort_index()

model_key
116     1
118     1
218     2
316     8
318    10
320    17
330     1
520     4
523     2
525     8
X1      5
X3      3
Name: count, dtype: int64

In [175]:
# Print value counts for model_key in the whole data restricted to the model_keys in the unusually cheap cars
data[data['model_key'].isin(unusually_cheap_cars['model_key'])]['model_key'].value_counts().sort_index()

model_key
116    358
118    143
218     13
316    234
318    568
320    752
330     33
520    633
523      4
525    184
X1     274
X3     438
Name: count, dtype: int64

In [176]:
# Print the mean price for each model_key in the unusually cheap cars
unusually_cheap_cars.groupby('model_key')['price'].mean().sort_index()

model_key
116    100.000000
118    800.000000
218    800.000000
316    687.500000
318    610.000000
320    682.352941
330    800.000000
520    625.000000
523    800.000000
525    500.000000
X1     760.000000
X3     566.666667
Name: price, dtype: float64

In [177]:
# Print the mean price for each model_key in the whole data restricted to the model_keys in the unusually cheap cars
data[data['model_key'].isin(unusually_cheap_cars['model_key'])].groupby('model_key')['price'].mean().sort_index()

model_key
116     9653.910615
118     9647.552448
218    17715.384615
316    10420.512821
318    11004.753521
320    12161.702128
330    16672.727273
520    15709.162717
523     6875.000000
525    15363.043478
X1     12703.649635
X3     18175.799087
Name: price, dtype: float64

Not removing cars with prices starting at 800 also prevents removing models with very low car count like 218 and 523 from the data set.

In [178]:
# Remove observations with price < 800
data = data[data['price'] >= 800]

In [179]:
data[data['price'] < 800]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,age_in_months_when_sold


## Visually explore the data

In [180]:
from src.plots import plot_avg_target_time_series_by_features, plot_distribution_of_features, plot_distribution_of_target_by_feature

In [181]:
# Get a list of columns in the data
data_columns = data.columns

In [182]:
car_features = [col for col in data_columns if 'feature' in col]

### Check price evolution according to categorical feature values

In [183]:
plot_avg_target_time_series_by_features(data, car_features)

In [109]:
small_cardinality_features = ['fuel', 'car_type', 'paint_color']

In [110]:
plot_avg_target_time_series_by_features(data, small_cardinality_features)

### Check numerical features distributions

In [111]:
features_for_distribution_plot = ['mileage', 'engine_power', 'age_in_months_when_sold', 'price']

In [112]:
plot_distribution_of_features(data, features_for_distribution_plot)

There are some extreme and odd values for these variables. Examples are:
- 1M miles drove. This can be rare but possible for old cars. 

In [70]:
plot_distribution_of_target_by_feature(data, 'model_key')

In [37]:
# Count number of observations by model_key
print(data['model_key'].value_counts().sort_values(ascending=False))

model_key
320                  752
520                  633
318                  569
X3                   438
116                  358
                    ... 
220 Active Tourer      1
635                    1
M135                   1
225 Active Tourer      1
214 Gran Tourer        1
Name: count, Length: 75, dtype: int64


In [77]:
# Make buckets for engine power
data['engine_power_bucket'] = pd.cut(data['engine_power'], bins=10)

In [78]:
data['engine_power_bucket']

0        (84.6, 126.9]
1       (296.1, 338.4]
2        (84.6, 126.9]
3       (126.9, 169.2]
4       (126.9, 169.2]
             ...      
4838     (84.6, 126.9]
4839     (84.6, 126.9]
4840     (84.6, 126.9]
4841     (84.6, 126.9]
4842    (126.9, 169.2]
Name: engine_power_bucket, Length: 4843, dtype: category
Categories (10, interval[float64, right]): [(-0.423, 42.3] < (42.3, 84.6] < (84.6, 126.9] < (126.9, 169.2] ... (253.8, 296.1] < (296.1, 338.4] < (338.4, 380.7] < (380.7, 423.0]]

TODO:
- make var that takes into account the series of the model
- check data counts for other variables
- think of an appropriate model


## Save cleaned data

In [71]:
from src.paths import CLEANED_DATA_DIR

In [ ]:
data.to_csv(CLEANED_DATA_DIR / 'cleaned_data.csv', index=False)